# Fluxo ETL de dados da camada Bronze para camada Silver

## Introdução

Na camada Bronze, os dados brutos, ou seja, sem tratamento e limpeza de dados estão armazenados no arquivo `TMDB_movie_dataset_v11.csv`. Nessa pipeline, serão realizadas as seguintes operações:

- **Limpar e padronizar os dados**: tratar valores nulos, duplicados e NaN;
- **Normalizar formatos**: datas, categorias e colunas numéricas;
- **Enriquecer ou derivar novas colunas** quando necessário para análises futuras;
- **Garantir a qualidade dos dados** antes do carregamento na Silver, que terá dados mais estruturados e prontos para consumo analítico.

Na camada Silver ficam armazenados os **dados tratados e consistentes**, que poderão ser utilizados em análises exploratórias e os dashboards no Tableau e PowerBI.  

As etapas do fluxo ETL são:

1. Carregamento dos dados Bronze (CSV bruto);
2. Inspeção e validação inicial das colunas;
3. Explosão dos campos compostos em linhas
4. Remoção de colunas que não serão usadas;
5. Tratamento de valores nulos, vazios, duplicados e NaN;
6. Tratamentos e remoção de outliers;
7. Armazenamento dos dados tratados na camada Silver em csv;
8. Carregamento no banco de dados.


## Leitura do csv bruto da camada Bronze

### Import das bibliotecas, configurações iniciais e inicialização da sessão spark

In [60]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, arrays_zip, col, trim, ltrim, lit, count, isnan
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, BooleanType, DateType
import time, os, shutil

# Inicio sessão spark
spark = SparkSession.builder.appName("tmdbEtlBronzeToSilver").getOrCreate()

### 1. Leitura do csv Bruto

Definição do schema do CSV para importação no pyspark dataframe e leitura do arquivo.

In [ ]:
# Setando o schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("vote_average", FloatType(), True),
    StructField("vote_count", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("release_date", DateType(), True),
    StructField("revenue", IntegerType(), True),
    StructField("runtime", IntegerType(), True),
    StructField("adult", BooleanType(), True),
    StructField("backdrop_path", StringType(), True),
    StructField("budget", IntegerType(), True),
    StructField("homepage", StringType(), True),
    StructField("imdb_id", StringType(), True),
    StructField("original_language", StringType(), True),
    StructField("original_title", StringType(), True),
    StructField("overview", StringType(), True),
    StructField("popularity", FloatType(), True),
    StructField("poster_path", StringType(), True),
    StructField("tagline", StringType(), True),
    StructField("genres", StringType(), True),
    StructField("production_companies", StringType(), True),
    StructField("production_countries", StringType(), True),
    StructField("spoken_languages", StringType(), True)
])

# Ingestão do arquivo
csv_path = "./bronze/TMDB_movie_dataset_v11.csv" 
df = spark.read.csv(csv_path, header=True, schema=schema, sep=',', quote='"', escape='"')
df.printSchema()

In [ ]:
df.summary().show(truncate=False)

### 2. Tratamento de campos compostos

No dataset de filmes existem 4 colunas com dados compostos, conforme indicado no [Dicionário de dados](https://github.com/Eric-chagas/film-data-analytics/blob/main/bronze/dicionario_dados.pdf). Essas colunas consistem em arrays com informações separadas por vírgula.

Cada linha nessas colunas, serão explodidas em uma ou várias linhas na dataframe principal interligadas pelo ID do filme `movie_id`, abaixo estão os nomes das colunas antes da explosão, e após:

1. `genres` -> `genre`
2. `production_companies` -> `production_company`
3. `production_countries` -> `production_country`
4. `spoken_languages` -> `spoken_language`

In [ ]:
df_split = df.withColumn("genre", split(col("genres"), ",")) \
                .withColumn("production_company", split(col("production_companies"), ",")) \
                .withColumn("production_country", split(col("production_countries"), ",")) \
                .withColumn("spoken_language", split(col("spoken_languages"), ","))

# print('Colunas split:')
# df_split.select("genre").distinct().show(truncate=False)
# df_split.select("production_company").distinct().show(truncate=False)
# df_split.select("production_country").distinct().show(truncate=False)
# df_split.select("spoken_language").distinct().show(truncate=False)


df_exploded = df_split.withColumn("genre", explode(col('genre'))) \
                        .withColumn("genre", trim(col("genre"))) \
                        .withColumn("production_company", explode(col("production_company"))) \
                        .withColumn("production_company", trim(col("production_company"))) \
                        .withColumn("production_country", explode(col("production_country"))) \
                        .withColumn("production_country", trim(col("production_country"))) \
                        .withColumn("spoken_language", explode(col("spoken_language"))) \
                        .withColumn("spoken_language", trim(col("spoken_language"))) \
                            
print("Colunas exploded:")             
df_exploded.select("id", "genre").distinct().show(truncate=False)
df_exploded.select("id","production_company").distinct().show(truncate=False)
df_exploded.select("id","production_country").distinct().show(truncate=False)
df_exploded.select("id","spoken_language").distinct().show(truncate=False)

old_cols = ["genres", "production_companies", "production_countries", "spoken_languages"]
df_exploded = df_split.drop(*old_cols)

### 3. Remoção das colunas que não serão usadas

Existem alguns campos no dataset que não trazem grande valor para a análise realizada nesse trabalho, por tanto, serão removidas. As colunas removidas, em sua maioria consistem em strings com URLs/Paths para imagens ou recursos externos relacionados ao filme, e são elas:

1. `backdrop_path`
2. `homepage`
3. `poster_path`

A descrição de cada um pode ser encontrada no [Dicionário de dados](https://github.com/Eric-chagas/film-data-analytics/blob/main/bronze/dicionario_dados.pdf).

In [36]:
# print(len(df_exploded.columns))

columns_to_drop = ["backdrop_path", "homepage", "poster_path", "imdb_id"]
df_refined_cols = df_exploded.drop(*columns_to_drop)

# print(len(df_refined_cols.columns))
# df_refined_cols.show(truncate=False)

### 4. Tratamento de valores nulos, vazios, duplicados e NaN

As regras de negócio aplicadas são:

1. Remover linhas com `revenue` null/NaN/None
2. Remover linhas com `release_date` null/NaN/None/NaT
3. Remover colunas 100% nulas caso existam
4. Remover linhas em que o título original e o título do filme `original_title` e `title` são ambos vazios
5. Remover linhas idênticas caso existam

In [ ]:
# print(df.select("Revenue").filter(col("Revenue").isNull()).count())
# print(df.select("Revenue").filter(isnan(col("Revenue"))).count())
# print(df.select("release_date").filter(col("release_date").isNull()).count())
print(f"df_cols original = {len(df_refined_cols.columns)}")

df_treated_nulls = df_refined_cols.dropna(how="all", subset=df_refined_cols.columns) 
print(f"df_cols nulls = {len(df_treated_nulls.columns)}")

df_treated_nulls = df_treated_nulls.filter(col("Revenue").isNotNull() & ~isnan(col("Revenue")))
print(f"df_rows revenue = {df_treated_nulls.count()}")

df_treated_nulls = df_treated_nulls.filter(col("release_date").isNotNull())
print(f"df_rows date = {df_treated_nulls.count()}")

df_treated_nulls = df_treated_nulls.filter((col("original_title") != "") | (col("title") != ""))
print(f"df_rows titles = {df_treated_nulls.count()}")

df_treated_nulls = df_treated_nulls.distinct()
print(f"df_rows lines = {df_treated_nulls.count()}")

### 4. Tratamentos e remoção de outliers

Os tratamentos realizados são:

1. Remoção de filmes classificados como "Adulto"
2. Remoção de outliers de popularidade


In [56]:
print(f"df_rows original = {df_treated_nulls.count()}")

df_output = df_treated_nulls.filter(col("adult") == False) 
print(f"df_cols no adult = {df_output.count()}")

df_output = df_output.filter(col("popularity") < 2000) 
print(f"df_cols no pop outlier = {df_output.count()}")

df_rows original = 1042016


df_cols no adult = 929278


df_cols no pop outlier = 929276


### 5. Armazenamento dos dados tratados na camada Silver em csv

Os dados tratados são armazenados na camada silver no formato `SILVER_TMDB_movie_dataset_v11_{data_horario}.csv`.

In [ ]:
time.tzset()
date_formatted = time.strftime("%Y%m%d")
time_formatted = time.strftime("%H%M%S")
timestamp_str = time.strftime("%Y-%m-%d %H:%M:%S")

print("Initiating csv save on silver...")

dest_filename = f"silver/SILVER_TMDB_movie_dataset_v11_{date_formatted}_{time_formatted}"

df_output.selectExpr([f"CAST({col} AS STRING)" for col in df_output.columns]).coalesce(1).write.mode("overwrite").option("header", "true").option("delimiter", ",").csv(dest_filename)

current_dir = os.getcwd()
files = os.listdir(f"{current_dir}/{dest_filename}")

try:
    for f in files:
        curr_file = f"{current_dir}/{dest_filename}/{f}"
        if f.endswith(".csv"):
            print(f"achei {curr_file}")
            shutil.copyfile(f"{curr_file}", f"{current_dir}/{dest_filename}.csv")
            os.remove(f"{curr_file}")
        else:
            os.remove(f"{curr_file}")
            
    os.rmdir(f"{current_dir}/{dest_filename}")
    print("Finished.")

except:
    print("Failed to extract csv.")


### 6. Inserção dos dados tratados no banco

O banco de dados é executado em um container Docker já deve estar rodando e disponível localmente na porta 5432. 

A conexão com o banco é feita utilizando uma string JDBC e o schema é criado dinâmicamente pelo spark, já de acordo com a estrutura do dataframe de saída `df_output`